In [ ]:
#https://www.kaggle.com/c/dont-call-me-turkey 



 This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import json
with open("../input/train.json") as f :
    df_train = pd.DataFrame(json.load(f))
with open("../input/test.json") as f :
    df_test = pd.DataFrame(json.load(f))
print("train data:", df_train.shape)
print("test data:", df_test.shape)

print("data columns")
print("train: ", list(df_train.columns))
print("test: ", list(df_test.columns))
max_frame_seq = df_train['audio_embedding'].apply(lambda x : len(x)).max()
print("max frame sequence : ", max_frame_seq)

print("is not turkey : ", df_train[df_train['is_turkey'] == 0]['is_turkey'].count())
print("is turkey : ", df_train[df_train['is_turkey'] == 1]['is_turkey'].count())

In [ ]:
import re
import lxml
from lxml import etree
import requests
from dask import bag

def get_title(vid) :
    try : return etree.HTML(requests.get("http://www.youtube.com/watch?v=" + vid).text).xpath("//span[@id='eow-title']/@title")[0]
    except : return ''
df_train['title'] = list(bag.from_sequence(df_train['vid_id'].values).map(get_title))
df_test['title'] = list(bag.from_sequence(df_test['vid_id'].values).map(get_title))

In [ ]:
def process_title(title) :
    title = re.sub('[^ A-Za-z]', '', title)
    title = title.strip()
    title = title.lower()
    return title
df_train['title'] = list(bag.from_sequence(df_train['title'].values).map(process_title))
df_test['title'] = list(bag.from_sequence(df_test['title'].values).map(process_title))

In [ ]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
train, validate = train_test_split(df_train)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

df_all = pd.DataFrame()
df_all['title'] = pd.concat([df_train['title'], df_test['title']])
tokenizer = Tokenizer(num_words = 20)
tokenizer.fit_on_texts(list(df_all['title'].values))
stopWords = set(stopwords.words('english'))
low_count_words = [w for w,c in tokenizer.word_counts.items() if c < 5 or w in stopWords]
for w in low_count_words:
    del tokenizer.word_index[w]
    del tokenizer.word_docs[w]
    del tokenizer.word_counts[w]
train_title    = tokenizer.texts_to_sequences(train['title'])
validate_title = tokenizer.texts_to_sequences(validate['title'])
test_title     = tokenizer.texts_to_sequences(df_test['title'])
train_train_title = tokenizer.texts_to_sequences(df_train['title'])

word_index = tokenizer.word_index
word_num = len(word_index)
print(word_num)

max_words = max(max([len(title) for title in train_train_title]), max([len(title) for title in test_title]))
print(max_words)

train_title    = pad_sequences(train_title, maxlen=max_words)
validate_title = pad_sequences(validate_title, maxlen=max_words)
test_title     = pad_sequences(test_title, maxlen=max_words)
train_train_title = pad_sequences(train_train_title, maxlen=max_words)

In [ ]:
from sklearn.model_selection import train_test_split

train, validate = train_test_split(df_train)
train_x = train['audio_embedding'].values
train_y = train['is_turkey'].values

validate_x = validate['audio_embedding'].values
validate_y = validate['is_turkey'].values

train_train_x = df_train['audio_embedding'].values
train_train_y = df_train['is_turkey'].values

test_x = df_test['audio_embedding'].values

from keras.preprocessing.sequence import pad_sequences

train_x       = pad_sequences(train_x, maxlen = max_frame_seq) / 255
validate_x    = pad_sequences(validate_x, maxlen = max_frame_seq) / 255
train_train_x = pad_sequences(train_train_x, maxlen = max_frame_seq) / 255
test_x        = pad_sequences(test_x, maxlen = max_frame_seq) / 255

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, LSTM, Bidirectional, Dropout, BatchNormalization, Flatten, concatenate, Activation, Embedding, CuDNNLSTM, GRU, CuDNNGRU, Embedding
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

dropout = 0.4
embedding_size = 3
inp_word = Input(shape=(max_words, ))
embedding = Embedding(word_num, embedding_size, input_length = max_words)(inp_word)
x1 = BatchNormalization()(embedding)
x1 = Bidirectional(CuDNNGRU(128, return_sequences=True))(x1)
x1 = Attention(max_words)(x1)

inp = Input(shape=(max_frame_seq, 128))
x2 = BatchNormalization(input_shape=(10, 128))(inp)
x2 = Bidirectional(CuDNNGRU(128, return_sequences=True))(x2)
x2 = Attention(max_frame_seq)(x2)

x = concatenate([x1, x2])
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=[inp_word, inp], outputs=x)
adam = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath="improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
best_epoch = 0

In [ ]:
#model.fit([train_title, train_x], train_y, batch_size=512, epochs=100, validation_data=([validate_title, validate_x], validate_y), callbacks=callbacks_list, verbose=0)

In [ ]:
"""
#fit on a portion of the training data, and validate on the rest
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, min_lr=1e-8)
# reduce_lr1 = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.1, patience=10, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=20,  restore_best_weights=True)

# History  = model.fit(x_train, y_train,batch_size=32,epochs=100,validation_data=(x_val, y_val),callbacks=[reduce_lr,early_stop])
History  = model.fit(x_train, y_train,batch_size=512, epochs=16,validation_data=[x_val, y_val],verbose = 2,callbacks=[reduce_lr,early_stop])
eva_plot(History)
# Get accuracy of model on validation data. It's not AUC but it's something at least!
score, acc = model.evaluate(x_val, y_val, batch_size=256)
print('Test accuracy:', acc)
"""

In [ ]:
model.fit([train_train_title, train_train_x], train_train_y, batch_size=512, epochs=21)

In [ ]:
predict = model.predict([test_title, test_x])
submission = pd.DataFrame()
submission['vid_id'] = df_test['vid_id']
submission['is_turkey'] = predict
submission.to_csv('csv_to_submit.csv', index = False)
print(submission.head())

In [ ]:
"""
import tensorflow as tf
import random
from sklearn.metrics import accuracy_score

enc_layer_size = [128]
enc_layer_num = len(enc_layer_size)

tf.reset_default_graph()
tf_dropout = tf.placeholder(shape = None, dtype = tf.float32)
seq = tf.placeholder(shape = None, dtype = tf.int32)

x = tf.placeholder(tf.float32, [None, max_frame_seq, 128])  
target = tf.placeholder(tf.float32, [None, 1])
inp = tf.layers.batch_normalization(x)
fw_enc_cells = [tf.contrib.rnn.LSTMCell(
                                            num_units = layer_size
                                            , activation = tf.nn.relu
                                            )
                    for layer_size in enc_layer_size]
fw_enc_cells = [tf.contrib.rnn.DropoutWrapper(
                                            enc_cell
                                            , output_keep_prob = 1.0 - tf_dropout
                                            , state_keep_prob = 1.0 - tf_dropout
                                            ) for enc_cell in fw_enc_cells]
fw_enc_cells = tf.contrib.rnn.MultiRNNCell(fw_enc_cells)
bw_enc_cells = [tf.contrib.rnn.LSTMCell(
                                            num_units = layer_size
                                            , activation = tf.nn.relu
                                            )
                    for layer_size in enc_layer_size]
bw_enc_cells = [tf.contrib.rnn.DropoutWrapper(
                                            enc_cell
                                            , output_keep_prob = 1.0 - tf_dropout
                                            , state_keep_prob = 1.0 - tf_dropout
                                            ) for enc_cell in bw_enc_cells]
bw_enc_cells = tf.contrib.rnn.MultiRNNCell(bw_enc_cells)
outputs, (fw_enc_state, bw_enc_state) = tf.nn.bidirectional_dynamic_rnn(fw_enc_cells, bw_enc_cells
                                            , inp, sequence_length = seq, time_major = False, dtype = tf.float32)
u_omega = tf.Variable(tf.random_normal([2], stddev=0.1))
output = tf.concat(outputs, 2)
#output = tf.sum(outputs, 2)
v = tf.nn.tanh(tf.layers.dense(inputs = output, units = 2))
vu = tf.tensordot(v, u_omega, axes=1, name='vu')
alphas = tf.nn.softmax(vu, name='alphas')
attention_vector = tf.reduce_sum(output * tf.expand_dims(alphas, -1), 1)
output = attention_vector

output = tf.layers.dense(
    inputs = output
    , units = 1
    , activation = None)

predict = tf.nn.sigmoid(output)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = target, logits = output))
        
learning_rate = tf.placeholder(shape = None, dtype = tf.float32)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

session = tf.Session()
session.run(tf.global_variables_initializer())

batch_size = 300
epoch = 1000
for i in range(epoch) :
    idx = [i for i in range(train_x.shape[0])]
    batch_idx = random.sample(idx, batch_size)
    feed_dict = {
        x : train_x[batch_idx]
        , target : train_y[batch_idx].reshape(-1, 1)
        , tf_dropout : 0.4
        , learning_rate : 0.001
        , seq : train_x_seq[batch_idx]
    }
    l, p, _ = session.run([loss, predict, optimizer], feed_dict = feed_dict)
    feed_dict = {
        x : validate_x
        , target : validate_y.reshape(-1, 1)
        , tf_dropout : 0.0
        , seq : validate_x_seq
    }
    l, p = session.run([loss, predict], feed_dict = feed_dict)
    acc = accuracy_score(validate_y == 1, p > 0.5)
    print(i, '\t', "{0:.4f}".format(l), '\t', "{0:.4f}".format(acc))
    
feed_dict = {
    x : test_x
    #, target : validate_y.reshape(-1, 1)
    , tf_dropout : 0.0
    , seq : test_x_seq
}

p = session.run([predict], feed_dict = feed_dict)
submission = pd.DataFrame()
submission['vid_id'] = df_test['vid_id']
submission['is_turkey'] = p[0]
submission.to_csv('csv_to_submit.csv', index = False)
print(submission)
"""